# Computer Vision III: Detection, Segmentation and Tracking (CV3DST)

- develop an extension of the ReID-based tracker that will make it more robust to occlusions by allowing it to recover from missed detections. 
- Adapt the track management scheme of our ReIDTracker allow it to recover from missed detections.


#### Install and import Python libraries

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm.autonotebook import tqdm

import torch
from torch.utils.data import DataLoader
from scipy.optimize import linear_sum_assignment as linear_assignment
import os.path as osp

import motmetrics as mm

mm.lap.default_solver = "lap"



In [ ]:

root_dir = ".."
sys.path.append(os.path.join(root_dir, "src"))

In [3]:

from mot.models.object_detector import FRCNN_FPN
from mot.data.data_track import MOT16Sequences
from mot.tracker.advanced import LongTermReIDHungarianTracker
from mot.utils import  cosine_distance
from mot.eval import run_tracker


/tmp/ipykernel_29561/123374960.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## Speed-Ups
In order to speed up training and inference runtimes, we will be working with pre-computed detections and ReID embeddings. We ran the object detector and applied to all frames. We also computed reid embeddings for all boxes in every frame of the dataset so that they don't need to be computed every time you run your tracker. This yields over 10x speed improvements

In [4]:
train_db = torch.load(
    osp.join(root_dir, "data/preprocessed_data/preprocessed_data_train_2.pth")
)


## ReIDHungarianTracker

our baseline is ReIDTracker.

This tracker works by performing frame-to-frame bipartite matching between newly detected boxes and past tracks based on ReID distance. Whenever a past track cannot be matched, its killed. And whenever, a newly detected box cannot be match, it starts a new trajectory.

We have modified the ``compute_distance`` function in ``data_association`` to include a thresshold on ReID distance (if ReID distance >0.1, matching is not possible). This is important to prevent our tracker from reusing tracks for very dissimilar objects.

Results:
```
          IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML   FP    FN IDs   FM  MOTA  MOTP
MOT16-02 40.5% 57.5% 31.3% 52.2% 96.1%  62  12  38 12  390  8873 203  210 49.1% 0.090
MOT16-05 54.4% 64.4% 47.1% 68.8% 94.0% 133  54  67 12  305  2156 330  149 59.7% 0.142
MOT16-09 49.9% 61.7% 41.9% 66.3% 97.7%  26  13  12  1   82  1793  77   76 63.3% 0.082
MOT16-11 61.1% 67.4% 55.9% 80.2% 96.6%  75  44  24  7  266  1871 176   91 75.5% 0.083
OVERALL  49.6% 62.3% 41.2% 63.5% 96.1% 296 123 141 32 1043 14693 786  526 59.0% 0.097
```


In [5]:
val_sequences = MOT16Sequences(
    "MOT16-reid", root_dir=osp.join(root_dir, "data/MOT16"), vis_threshold=0.0
)


In [6]:
tracker = ReIDHungarianTracker(obj_detect=None)
run_tracker(val_sequences, db=train_db, tracker=tracker, output_dir=None)


NameError: name 'ReIDHungarianTracker' is not defined

## Long-Term ReID Tracker


The tracker above has an obvious limitation: whenever a track cannot be matched with the detections of a given frame the track will be killed. This means that if our detector misses an object in a single frame (due to e.g. occlusion), we will not be able to recover that track, and we will start a new one. 

To fix this issue, we would like to allow our tracker to maintain tracks that are not matched during data association. We will refer to these tracks as **inactive**. During data association, we will try to match the detected boxes for the current frame to both tracks that are active (i.e. tracks that we were able to match in the previous frame) as well as those that are inactive. Therefore, if a detector misses an object in a frame and the object reappears after a few frames, we will still be able to match it to its corresponding track, instead of creating a new one.

In order to adapt our tracker to have this behavior, we will use the `inactive` attribute from the `track` class (see `tracker/tracker.py`. This attribute will be assigned an integer indicating for how many frames a track has remained unmatched. Whenever we are able to match the track `t`, we will set `t.inactive=0` and, naturally, when tracks are initialized, the class constructor sets `inactive=0`. 

Your job is to maintain the `inactive` attribute of all tracks being kept by tracker so that its value represents the number of frames for which the track has been unmatched. Additionally, we introduce a `patience` parameter. Whenever a track has been inactive for more than `inactive` frames. it will need to be killed.

Results should approximately be around:

```
          IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML   FP    FN IDs   FM  MOTA  MOTP
MOT16-02 45.9% 65.1% 35.4% 52.2% 96.1%  62  12  37 13  390  8873 130  210 49.4% 0.090
MOT16-05 63.4% 75.0% 54.9% 68.8% 94.0% 133  54  67 12  305  2156 283  149 60.3% 0.142
MOT16-09 52.5% 64.9% 44.1% 66.3% 97.7%  26  13  12  1   82  1793  49   76 63.9% 0.083
MOT16-11 68.3% 75.3% 62.5% 80.2% 96.6%  75  44  24  7  266  1871 136   90 75.9% 0.083
OVERALL  55.7% 70.0% 46.2% 63.5% 96.1% 296 123 140 33 1043 14693 598  525 59.4% 0.097
```


In [ ]:
tracker = LongTermReIDHungarianTracker(patience=20, obj_detect=None)
run_tracker(val_sequences, db=train_db, tracker=tracker, output_dir=None)


Tracking: MOT16-02
Tracks found: 207
Runtime for MOT16-02: 0.9 s.
Tracking: MOT16-05
Tracks found: 355
Runtime for MOT16-05: 0.9 s.
Tracking: MOT16-09
Tracks found: 65
Runtime for MOT16-09: 0.5 s.
Tracking: MOT16-11
Tracks found: 217
Runtime for MOT16-11: 1.0 s.
Runtime for all sequences: 3.4 s.
          IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML   FP    FN IDs   FM  MOTA  MOTP IDt IDa IDm
MOT16-02 45.9% 65.1% 35.4% 52.2% 96.1%  62  12  37 13  390  8873 130  210 49.4% 0.090  18 113   5
MOT16-05 63.4% 75.0% 54.9% 68.8% 94.0% 133  54  67 12  305  2156 283  149 60.3% 0.142  73 216  13
MOT16-09 52.5% 64.9% 44.1% 66.3% 97.7%  26  13  12  1   82  1793  49   76 63.9% 0.083  12  40   4
MOT16-11 68.3% 75.3% 62.5% 80.2% 96.6%  75  44  24  7  266  1871 136   90 75.9% 0.083  23 115   3
OVERALL  55.7% 70.0% 46.2% 63.5% 96.1% 296 123 140 33 1043 14693 598  525 59.4% 0.097 126 484  25


,idf1,idp,idr,recall,precision,num_unique_objects,mostly_tracked,partially_tracked,mostly_lost,num_false_positives,num_misses,num_switches,num_fragmentations,mota,motp,num_transfer,num_ascend,num_migrate
MOT16-02,0.458524,0.651119,0.353856,0.522469,0.961378,62,12,37,13,390,8873,130,210,0.494484,0.090339,18,113,5
MOT16-05,0.633898,0.749704,0.549082,0.688304,0.939795,133,54,67,12,305,2156,283,149,0.603296,0.141942,73,216,13
MOT16-09,0.525115,0.649419,0.440751,0.663286,0.977310,26,13,12,1,82,1793,49,76,0.638685,0.082643,12,40,4
MOT16-11,0.683269,0.753288,0.625159,0.801717,0.966032,75,44,24,7,266,1871,136,90,0.759114,0.082754,23,115,3
OVERALL,0.556888,0.699726,0.462480,0.635038,0.960803,296,123,140,33,1043,14693,598,525,0.594277,0.096641,126,484,25
